# Data Cleaning dan Analisis Sweetviz

In [10]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_json("fashion_boutique_dataset.json")
df.head(5)

,product_id,category,brand,season,size,color,original_price,markdown_percentage,current_price,purchase_date,stock_quantity,customer_rating,is_returned,return_reason
0,FB000001,Outerwear,Zara,Spring,XL,Red,196.01,0.0,196.01,2025-07-05T00:00:00.000,37,3.0,False,None
1,FB000002,Tops,Uniqlo,Winter,L,Pink,119.64,0.0,119.64,2025-08-06T00:00:00.000,2,2.5,False,None
2,FB000003,Accessories,Uniqlo,Winter,None,Black,33.80,0.0,33.80,2025-08-06T00:00:00.000,22,4.3,False,None
3,FB000004,Shoes,Uniqlo,Spring,XL,Black,75.36,0.0,75.36,2025-07-07T00:00:00.000,48,2.6,False,None
4,FB000005,Tops,Banana Republic,Winter,XL,Black,105.02,0.0,105.02,2025-08-06T00:00:00.000,10,NaN,False,None


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           2176 non-null   object 
 1   category             2176 non-null   object 
 2   brand                2176 non-null   object 
 3   season               2176 non-null   object 
 4   size                 1685 non-null   object 
 5   color                2176 non-null   object 
 6   original_price       2176 non-null   float64
 7   markdown_percentage  2176 non-null   float64
 8   current_price        2176 non-null   float64
 9   purchase_date        2176 non-null   object 
 10  stock_quantity       2176 non-null   int64  
 11  customer_rating      1814 non-null   float64
 12  is_returned          2176 non-null   bool   
 13  return_reason        320 non-null    object 
dtypes: bool(1), float64(4), int64(1), object(8)
memory usage: 223.3+ KB


In [12]:
# Pastikan kolom 'customer_rating' bertipe numerik dan isi NaN dengan rata-rata
if df['customer_rating'].dtype != 'float':
    df['customer_rating'] = pd.to_numeric(df['customer_rating'], errors='coerce')
df['customer_rating'].fillna(df['customer_rating'].mean(), inplace=True)
# Kolom 'size' tetap diisi dengan '-'
df['size'].fillna('-', inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_9956\2640498124.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['customer_rating'].fillna(df['customer_rating'].mean(), inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_9956\2640498124.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [14]:
# ===============================
# 1. DROP UNNEEDED COLUMN
# ===============================
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

In [15]:

# ===============================
# 2. FIX PURCHASE DATE
# ===============================
df['purchase_date'] = pd.to_datetime(df['purchase_date'], errors='coerce')

df['year'] = df['purchase_date'].dt.year
df['month'] = df['purchase_date'].dt.month
df['day'] = df['purchase_date'].dt.day
df['day_name'] = df['purchase_date'].dt.day_name()


In [16]:
# ===============================
# 3. FIX CUSTOMER RATING
# ===============================
df['customer_rating'] = pd.to_numeric(df['customer_rating'], errors='coerce')

# valid rating 1–5 only
df.loc[(df['customer_rating'] < 1) | (df['customer_rating'] > 5), 'customer_rating'] = np.nan

# optional: isi rating kosong pakai median
df['customer_rating'].fillna(df['customer_rating'].median(), inplace=True)


C:\Users\User\AppData\Local\Temp\ipykernel_9956\2535359997.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['customer_rating'].fillna(df['customer_rating'].median(), inplace=True)


In [18]:
# ===============================
# 4. CLEAN SIZE COLUMN
# ===============================
df['size'] = df['size'].replace('-', np.nan)
df['size'] = df['size'].fillna('Unknown')


In [19]:
# ===============================
# 5. CLEAN RETURN REASON
# ===============================
df['return_reason'] = np.where(
    df['is_returned'] == True,
    df['return_reason'],
    'No Return'
)


In [20]:
# ===============================
# 6. VALIDATE MARKDOWN LOGIC
# ===============================
# markdown tidak boleh negatif
df.loc[df['markdown_percentage'] < 0, 'markdown_percentage'] = 0

# markdown tidak boleh lebih besar dari original_price
df.loc[df['markdown_percentage'] > df['original_price'], 'markdown_percentage'] = 0


In [21]:
# ===============================
# 7. TEXT NORMALIZATION
# ===============================
text_cols = ['category', 'brand', 'season', 'color']

for col in text_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.title()
    )


In [22]:
# ===============================
# 8. STOCK VALIDATION
# ===============================
df.loc[df['stock_quantity'] < 0, 'stock_quantity'] = 0
df['out_of_stock'] = np.where(df['stock_quantity'] == 0, 1, 0)


In [23]:
# ===============================
# 9. REMOVE DUPLICATES
# ===============================
df.drop_duplicates(subset=['product_id', 'purchase_date'], inplace=True)


In [24]:
# ===============================
# 10. FEATURE ENGINEERING
# ===============================
df['discount_flag'] = np.where(df['markdown_percentage'] > 0, 1, 0)

df['rating_category'] = pd.cut(
    df['customer_rating'],
    bins=[0, 2, 4, 5],
    labels=['Low', 'Medium', 'High']
)


In [25]:
# ===============================
# SAVE CLEAN DATA
# ===============================
df.to_csv("clean_data_final.csv", index=False)

print("✅ Data cleaning selesai. File saved: clean_data_final.csv")

✅ Data cleaning selesai. File saved: clean_data_final.csv


In [26]:
#sweetviz
import sweetviz as sv
quick_analyst = sv.analyze(df)
quick_analyst.show_html("SWEETVIZ_REPORT.html")

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:01 -> (00:00 left)


Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [27]:
#ydata_profiling
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Laporan Analisis Data")
profile.to_file("laporan_eda.html")

Summarize dataset:  82%|████████▏ | 23/28 [00:01<00:00, 13.13it/s, Calculate auto correlation]        C:\Users\User\Documents\BELAJAR DATA ANALYST\Retail Fashion Boutique Data Sales Analytics 2025\.venv\Lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[9 9 9 ... 9 9 9]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(
C:\Users\User\Documents\BELAJAR DATA ANALYST\Retail Fashion Boutique Data Sales Analytics 2025\.venv\Lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[5 6 6 ... 6 6 6]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discr